In [ ]:
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

import numpy as np
from tqdm import tqdm
from skimage.draw import line
import matplotlib.pyplot as plt

In [ ]:
UPSCALE_FACTOR = 8
N_POINTS = 256
ITERATIONS = 10001
INTERVAL = 100

RANDOM_SCALE = 10

In [ ]:
# Load and convert a square image to a circular cropped, grayscale image
img = Image.open("ada.png")
img = img.resize((img.width * UPSCALE_FACTOR, img.height * UPSCALE_FACTOR))
img = img.convert("L")
img = np.array(img)

radius = img.shape[0] // 2
yy, xx = np.ogrid[:img.shape[0], :img.shape[1]]
circle = (xx - radius) ** 2 + (yy - radius) ** 2 <= radius ** 2
img[~circle] = 0

img = img.astype(np.float32) / 255.0

Image.fromarray(np.uint8(img*255))

In [ ]:
# DEFINE THREADS

n_connections = N_POINTS * (N_POINTS - 1) // 2
print(n_connections, "threads")

string_img = img.copy()
width, height = img.shape
center = (width // 2, height // 2)


angles = np.linspace(0, 2 * np.pi, N_POINTS, endpoint=False) + np.random.rand(N_POINTS) / (2 * np.pi * RANDOM_SCALE)
points_x = np.int32(np.round(center[0] + radius * np.cos(angles)))
points_y = np.int32(np.round(center[1] + radius * np.sin(angles)))

connections = []

n_connections = 0
for i in tqdm(range(N_POINTS)):
    single_pin = []
    for j in range(N_POINTS):
        if i == j: continue

        x1, y1 = points_x[i], points_y[i]
        x2, y2 = points_x[j], points_y[j]

        rr, cc = line(y1, x1, y2, x2)
        valid = (rr >= 0) & (rr < height) & (cc >= 0) & (cc < width)
        rr, cc = rr[valid], cc[valid]
        if len(rr) < 2: continue
        x, y = cc, rr

        single_pin.append({
            "x": x,
            "y": y,
            "id": j
        })
        n_connections += 1

    n_connections_pin = len(single_pin)
    lengths = np.array([len(connection["x"]) for connection in single_pin], dtype=np.int32)
    max_len = max(lengths)
    
    x_coords = np.full((n_connections_pin, max_len), -1, dtype=np.int32)
    y_coords = np.full((n_connections_pin, max_len), -1, dtype=np.int32)

    for j in range(n_connections_pin):
        x_coords[j, :len(single_pin[j]["x"])] = single_pin[j]["x"]
        y_coords[j, :len(single_pin[j]["y"])] = single_pin[j]["y"]

    connections.append({
        "x": x_coords,
        "y": y_coords,
        "lengths": lengths,
        "n_connections_pin": n_connections_pin,
        "ids": np.array([connection["id"] for connection in single_pin], dtype=np.int32),
    })

In [ ]:
# CHECK

string_img = img.copy()
string_img[~circle] = 0

for i in range(connections[0]["n_connections_pin"]):
    x = connections[0]["x"][i, :connections[0]["lengths"][i]]
    y = connections[0]["y"][i, :connections[0]["lengths"][i]]
    string_img[y, x] = 1
Image.fromarray(np.uint8(string_img*255))

In [ ]:
# DRAW CYCLE
# https://www.cg.tuwien.ac.at/research/publications/2018/Birsak2018-SA/Birsak2018-SA-preprint.pdf
# https://gabrieleballetti.github.io/posts/2022-07-29-string-art/

string_img = np.ones_like(img)
string_img[~circle] = 0

wash_img = img.copy()

def save_image(image_array, filename):
    try: Image.fromarray(image_array).save(filename)
    except Exception as e: print(f"Error saving image {filename}: {e}")
def update_image(iteration, executor):
    image_data = np.uint8(string_img*255)
    filename = f"output/{iteration:05d}.png"
    executor.submit(save_image, image_data.copy(), filename)

import numba
from numba import jit, prange
@jit(nopython=True, parallel=True)
def compute_all_scores(wash_img, x_coords, y_coords, lengths, n_connections_pin):    
    scores = np.zeros(n_connections_pin, dtype=np.float32)
    for i in prange(n_connections_pin):
        length = lengths[i]
            
        total = 0.0
        for j in range(length):
            total += wash_img[y_coords[i, j], x_coords[i, j]]
        
        scores[i] = total / length
    
    return scores

opacity_multiplier = 0.85
starting_pin = 0

executor = ThreadPoolExecutor(max_workers=2)
score_history = np.zeros(ITERATIONS, dtype=np.float32)
pbar = tqdm(range(ITERATIONS))
for iteration in pbar:
    x_coords = connections[starting_pin]["x"]
    y_coords = connections[starting_pin]["y"]
    lengths = connections[starting_pin]["lengths"]
    n_connections_pin = connections[starting_pin]["n_connections_pin"]
    ids = connections[starting_pin]["ids"]

    scores = compute_all_scores(wash_img, x_coords, y_coords, lengths, n_connections_pin)

    best_thread = np.argmin(scores)
    min_score = scores[best_thread]
    score_history[iteration] = min_score
    
    x, y = x_coords[best_thread, :lengths[best_thread]], y_coords[best_thread, :lengths[best_thread]]
    
    string_img[y, x] = 0
    # 1st way: new = 1
    # 2nd way: new = old + opacity_multiplier * (1 - old) = opacity_multiplier + (1 - opacity_multiplier) * old
    wash_img[y, x] = opacity_multiplier + (1 - opacity_multiplier) * np.clip(wash_img[y, x], 0, 1)
    #wash_img[y, x] = 1
    # wash_img[y, x] = 0.85
    
    starting_pin = ids[best_thread]
    
    # Update display
    if iteration % INTERVAL == 0: update_image(iteration, executor)
    pbar.set_description(f"Iter {iteration}, score {min_score:.3f}")

plt.plot(score_history)
plt.ylabel('Grayscale score')
plt.xlabel('Iteration')
plt.ylim(0, 1)
plt.hlines(y=np.max(score_history), xmin=0, xmax=ITERATIONS, color="r", linestyle="--", label=f"Max score {np.max(score_history):.3f}")
plt.legend(loc="lower right")
plt.show()

executor.shutdown(wait=True)